In [10]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf("mock_trial.pdf")


In [11]:
def estimate_tokens(text):
    return len(text) // 4

total_tokens = estimate_tokens(pdf_text)
print(f"Estimated tokens: {total_tokens}")


Estimated tokens: 5215


In [12]:
judge_prompt = f"""
You are a judge presiding over a mock trial. You must adhere to the trial's context and answer questions as the judge.

Here is the trial document:
{pdf_text} 

You are ready to proceed. The user will be roleplaying as another character in the mock trial. 
"""


In [13]:
car_owner_prompt = f"""
You are pretending to be the car owner in this mock trial. You must adhere to the trial's context and answer questions as the car owner.

Here is the trial document:
{pdf_text} 

You are ready to proceed. The user will be roleplaying as another character in the mock trial. 
"""

In [14]:
import autogen
from autogen import ConversableAgent
import os


# llm_config = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY"), "api_rate_limit": 1.0}]}
llm_config = {"config_list": [{"model": "llama3.2:latest", "api_type": "ollama", "client_host": "http://127.0.0.1:11434"}]}

    
# the main entrypoint/supervisor agent
judge_agent = ConversableAgent("judge agent", system_message=judge_prompt, llm_config=llm_config)

In [15]:
car_owner_agent = ConversableAgent(name="car owner agent", system_message=car_owner_prompt, llm_config=llm_config)

In [16]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

In [17]:
from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[judge_agent, car_owner_agent, human_proxy],
    messages=[],
    max_round=6,
)


group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config 
)

In [ ]:
chat_result = human_proxy.initiate_chat(
    group_chat_manager,
    message="I will be roleplaying as the Deputy DA in this mock trial, is everyone ready?",
    summary_method="reflection_with_llm",
)

human_proxy (to chat_manager):

I will be roleplaying as the Deputy DA in this mock trial, is everyone ready?

--------------------------------------------------------------------------------

Next speaker: car owner agent


>>>>>>>> USING AUTO REPLY...
car owner agent (to chat_manager):

I'll play the role of the defendant's attorney. My client, John Smith, has been accused of embezzling funds from a local charity. I'm prepared to defend him against these serious charges.

Before we begin, let me just review the case files and make sure I have all the necessary materials.

Also, as the Deputy DA, please go ahead and proceed with opening statements. What would you like to say to the court about the evidence and the case?

--------------------------------------------------------------------------------

Next speaker: human_proxy

human_proxy (to chat_manager):

Why did you steal the car?

--------------------------------------------------------------------------------

Next speaker: jud

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",  # Choose model with sufficient token capacity
#     messages=[{"role": "system", "content": roleplay_prompt}]
# )

# print(completion.choices[0].message)


ChatCompletionMessage(content='Understood. I am prepared to answer questions within the context of this mock trial. Please proceed with your inquiries.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
